# Configuração de ambiente

Importação das bibliotecas:
- Pandas: para manipulação de dados através do alias "pd"
- Google Colab: para manipulação de arquivos no Google Drive

E execução do comando "%matplotlib inline" para que os gráficos sejam exibidos no notebook logo abaixo da célula de código que os gerou.

In [123]:
import pandas as pd
from google.colab import drive

%matplotlib inline

Carregamento do Drive para que o notebook possa acessar os arquivos do Google Drive e com o parâmetro "force_remount=True" para que o Drive seja montado a cada execução do notebook.

In [124]:
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


# Leitura e visualização do dataset

Leitura do arquivo "dataset.csv" através da função "read_csv" do Pandas e armazenamento do resultado na variável "df". Uso do parâmetro "sep" para informar qual o separador de colunas do arquivo e do parâmetro "encoding" para informar qual o tipo de codificação do arquivo.

In [125]:
csv_url = "https://github.com/viniciusgugelmin/data-science-3/blob/main/projects/valorant_stats/data/dataset.csv?raw=true"

df = pd.read_csv(csv_url, sep=',', encoding='utf-8', low_memory=False)

Atribuição do limite na exibição do dataset para 50 colunas e 20 linhas, afim de conseguir visualizar melhor os dados.

In [126]:
pd.set_option('display.max_columns', 50)
pd.set_option('display.max_rows', 20)

Exibição das 5 primeiras linhas do DataFrame através da função "head" do Pandas para verificar se o arquivo foi carregado corretamente e ter uma ideia do que ele contém.

In [127]:
df.head()

,region,name,tag,rating,damage_round,headshots,headshot_percent,aces,clutches,flawless,first_bloods,kills,deaths,assists,kd_ratio,kills_round,most_kills,score_round,wins,win_percent,agent_1,agent_2,agent_3,gun1_name,gun1_head,gun1_body,gun1_legs,gun1_kills,gun2_name,gun2_head,gun2_body,gun2_legs,gun2_kills,gun3_name,gun3_head,gun3_body,gun3_legs,gun3_kills
0,NaN,ShimmyXD,#NA1,Radiant,135.8,992,24.9,0,140,80,161,"1,506","1,408",703,1.07,0.7,29,208.8,59,59.6,Fade,Viper,Omen,Vandal,35,59,5,802,Phantom,33,62,5,220,Classic,36,60,3,147
1,NaN,XSET Cryo,#cells,Radiant,170.3,879,28.3,2,122,94,316,"1,608","1,187",206,1.35,1.0,32,270.6,52,65.8,Chamber,Jett,Raze,Vandal,41,56,3,689,Operator,8,91,0,226,Phantom,32,63,5,137
2,NaN,PuRelittleone,#yoruW,Radiant,147.5,720,24.0,3,117,59,216,"1,115","1,064",267,1.05,0.8,39,227.8,42,65.6,Yoru,Jett,Chamber,Vandal,38,57,4,444,Phantom,36,61,3,231,Operator,8,91,1,102
3,NaN,Boba,#0068,Radiant,178.2,856,37.3,3,83,49,235,"1,134",812,157,1.40,1.0,37,277.0,32,62.8,Jett,Chamber,KAY/O,Vandal,51,47,2,754,Sheriff,48,51,1,48,Phantom,44,56,0,36
4,NaN,i love mina,#kelly,Radiant,149.8,534,24.4,2,71,38,137,869,781,213,1.11,0.8,29,230.9,32,62.8,Jett,Raze,Chamber,Vandal,36,60,4,419,Spectre,21,71,8,65,Operator,8,92,0,64


# Processo inicial de limpeza

Apenas visualização das colunas

In [128]:
df.columns

Index(['region', 'name', 'tag', 'rating', 'damage_round', 'headshots',
       'headshot_percent', 'aces', 'clutches', 'flawless', 'first_bloods',
       'kills', 'deaths', 'assists', 'kd_ratio', 'kills_round', 'most_kills',
       'score_round', 'wins', 'win_percent', 'agent_1', 'agent_2', 'agent_3',
       'gun1_name', 'gun1_head', 'gun1_body', 'gun1_legs', 'gun1_kills',
       'gun2_name', 'gun2_head', 'gun2_body', 'gun2_legs', 'gun2_kills',
       'gun3_name', 'gun3_head', 'gun3_body', 'gun3_legs', 'gun3_kills'],
      dtype='object')

Antes de tudo resolvemos tirar jogadores que tenham menos de 20 vitórias, para assim evitarmos outliers como jogadores não ativos que acabam apenas jogando poucas partidas e estagnando no elo/ranque obtido, sabendo que para ser um jogador relevante ele acabará tendo muito mais que 20 vitórias, jogando ainda mais o jogo.

In [129]:
df = df[df['wins'] > 20]

Escolhemos tirar a região e etiqueta pois são dados não tão relevantes para esta pesquisa. Tiramos eliminações e vezes eliminadas pois já temos o percentual que é o cálculo que contém os dois números. Tiramos maior quantidade de eliminações e total de vitórias pois acaba sendo algo não tão relevante para o aprendizado, visto que os dados são bem parecidos na maioria dos casos e pode acabar mais confundindo do que ajudando. E por fim, tiramos dados de agentes e armas secundárias e terciárias pois não são tão relevantes.

In [130]:
df.drop(['region', 'tag', 'kills', 'deaths', 'most_kills', 'wins', 'agent_2', 'agent_3', 'gun1_kills'], inplace=True, axis=1)
df = df.drop(df.filter(regex='gun2_|gun3_|_head|_body|_legs').columns, axis=1)
df.head()

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4913: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,name,rating,damage_round,headshots,headshot_percent,aces,clutches,flawless,first_bloods,assists,kd_ratio,kills_round,score_round,win_percent,agent_1,gun1_name
0,ShimmyXD,Radiant,135.8,992,24.9,0,140,80,161,703,1.07,0.7,208.8,59.6,Fade,Vandal
1,XSET Cryo,Radiant,170.3,879,28.3,2,122,94,316,206,1.35,1.0,270.6,65.8,Chamber,Vandal
2,PuRelittleone,Radiant,147.5,720,24.0,3,117,59,216,267,1.05,0.8,227.8,65.6,Yoru,Vandal
3,Boba,Radiant,178.2,856,37.3,3,83,49,235,157,1.40,1.0,277.0,62.8,Jett,Vandal
4,i love mina,Radiant,149.8,534,24.4,2,71,38,137,213,1.11,0.8,230.9,62.8,Jett,Vandal


# Processo de limpeza de dados nulos e padronização de dados

Utilização do método **info** para tentar encontrar valores nulos que não deveriam ser nulos.

In [131]:
df_cleaned = df.copy()

df_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 41103 entries, 0 to 85677
Data columns (total 16 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   name              41103 non-null  object 
 1   rating            41103 non-null  object 
 2   damage_round      41103 non-null  float64
 3   headshots         41103 non-null  object 
 4   headshot_percent  41103 non-null  float64
 5   aces              41103 non-null  int64  
 6   clutches          41103 non-null  int64  
 7   flawless          41103 non-null  int64  
 8   first_bloods      41103 non-null  object 
 9   assists           41103 non-null  object 
 10  kd_ratio          41103 non-null  float64
 11  kills_round       41103 non-null  float64
 12  score_round       41103 non-null  float64
 13  win_percent       41103 non-null  float64
 14  agent_1           41103 non-null  object 
 15  gun1_name         41103 non-null  object 
dtypes: float64(6), int64(3), object(7)
memor

Acabamos percebendo que temos linhas com nomes de jogadores vazio, então resolvemos tirar mesmo que não seja algo relevante para o aprendizado supervisionado, acaba atrapalhando na comprovação e amostragem dos dados. Também retiramos colunas numéricas com valores igual a 0, que provavelmente são os nossos outliers.

In [132]:
df_cleaned = df_cleaned[df_cleaned['name'].notna()]
df_cleaned = df_cleaned[df_cleaned['headshots'] != "0"]
df_cleaned = df_cleaned[df_cleaned['headshot_percent'] != "0"]
df_cleaned = df_cleaned[df_cleaned['aces'] != "0"]
df_cleaned = df_cleaned[df_cleaned['clutches'] != "0"]
df_cleaned = df_cleaned[df_cleaned['flawless'] != "0"]
df_cleaned = df_cleaned[df_cleaned['first_bloods'] != "0"]
df_cleaned = df_cleaned[df_cleaned['assists'] != "0"]
df_cleaned = df_cleaned[df_cleaned['kd_ratio'] != "0"]
df_cleaned = df_cleaned[df_cleaned['kills_round'] != "0"]
df_cleaned = df_cleaned[df_cleaned['score_round'] != "0"]
df_cleaned = df_cleaned[df_cleaned['win_percent'] != "0"]
df_cleaned = df_cleaned[df_cleaned['win_percent'] != "0.0"]
df_cleaned = df_cleaned[df_cleaned['win_percent'] != "100"]
df_cleaned = df_cleaned[df_cleaned['win_percent'] != "100.0"]
df_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 41103 entries, 0 to 85677
Data columns (total 16 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   name              41103 non-null  object 
 1   rating            41103 non-null  object 
 2   damage_round      41103 non-null  float64
 3   headshots         41103 non-null  object 
 4   headshot_percent  41103 non-null  float64
 5   aces              41103 non-null  int64  
 6   clutches          41103 non-null  int64  
 7   flawless          41103 non-null  int64  
 8   first_bloods      41103 non-null  object 
 9   assists           41103 non-null  object 
 10  kd_ratio          41103 non-null  float64
 11  kills_round       41103 non-null  float64
 12  score_round       41103 non-null  float64
 13  win_percent       41103 non-null  float64
 14  agent_1           41103 non-null  object 
 15  gun1_name         41103 non-null  object 
dtypes: float64(6), int64(3), object(7)
memor

Notamos também que existem algumas colunas que apresentam apenas números mas estão no formato errado, então fazemos a conversão. Também aproveitamos para remover jogadores que estão com porcentagem de vitória muito alta e baixa, pois provavelmente não estão nos elos/ranques adequados, ainda podem subir ou cair muito por estar com essa porcentagem.

In [133]:
df_cleaned['headshots'] = pd.to_numeric(df_cleaned['first_bloods'].str.replace(",","."), errors="coerce")
df_cleaned['first_bloods'] = pd.to_numeric(df_cleaned['first_bloods'].str.replace(",","."), errors="coerce")
df_cleaned['assists'] = pd.to_numeric(df_cleaned['assists'].str.replace(",","."), errors="coerce")
df_cleaned = df_cleaned[df_cleaned['win_percent'] < 80]
df_cleaned = df_cleaned[df_cleaned['win_percent'] > 40]
df_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 40727 entries, 0 to 85677
Data columns (total 16 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   name              40727 non-null  object 
 1   rating            40727 non-null  object 
 2   damage_round      40727 non-null  float64
 3   headshots         40727 non-null  float64
 4   headshot_percent  40727 non-null  float64
 5   aces              40727 non-null  int64  
 6   clutches          40727 non-null  int64  
 7   flawless          40727 non-null  int64  
 8   first_bloods      40727 non-null  float64
 9   assists           40727 non-null  float64
 10  kd_ratio          40727 non-null  float64
 11  kills_round       40727 non-null  float64
 12  score_round       40727 non-null  float64
 13  win_percent       40727 non-null  float64
 14  agent_1           40727 non-null  object 
 15  gun1_name         40727 non-null  object 
dtypes: float64(9), int64(3), object(4)
memor

In [134]:
df_cleaned['rating'].unique()

array(['Radiant', 'Immortal 3', 'Immortal 2', 'Immortal 1', 'Diamond 3',
       'Gold 1', 'Silver 2', 'Platinum 1', 'Gold 3', 'Gold 2', 'Silver 1',
       'Platinum 2', 'Diamond 1', 'Silver 3', 'Bronze 3', 'Platinum 3'],
      dtype=object)

Identificamos também que temos certas divisões entre os elos que acaba sendo pouco relevante para estudo, visto que são bem pequenas as diferenças.

In [135]:
df_cleaned['rating'] = df_cleaned['rating'].str.replace(r'Immortal 3', 'Radiant', regex=True)
df_cleaned['rating'] = df_cleaned['rating'].str.replace(r'Immortal 1', 'Diamond -', regex=True)
df_cleaned['rating'] = df_cleaned['rating'].str.replace(r'^Immortal.*', 'Immortal', regex=True)
df_cleaned['rating'] = df_cleaned['rating'].str.replace(r'^Diamond.*', 'Diamond -', regex=True)
df_cleaned['rating'] = df_cleaned['rating'].str.replace(r'^Platinum.*', 'Diamond -', regex=True)
df_cleaned['rating'] = df_cleaned['rating'].str.replace(r'^Gold.*', 'Diamond -', regex=True)
df_cleaned['rating'] = df_cleaned['rating'].str.replace(r'^Silver.*', 'Diamond -', regex=True)
df_cleaned['rating'] = df_cleaned['rating'].str.replace(r'^Bronze.*', 'Diamond -', regex=True)

df_cleaned['rating'].unique()

array(['Radiant', 'Immortal', 'Diamond -'], dtype=object)

# Salvando dados limpos

Para salvar os dados limpos em um arquivo, vamos usar o método to_csv

In [136]:
df_cleaned.to_csv('/content/drive/MyDrive/UP/Ciência de Dados/Notebooks/data/dataset_clean.csv', index=False, sep=';')